In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import ast

## Train-test split and encoding for physiochemical

In [3]:
fe_df = pd.read_csv("../../dataset/scooter_features/output_results.csv")
fe_df

,Original,AAC,DC,molecular_weight,isoelectric_point,gravy,aromaticity,instability_index,aliphatic_index,boman_index
0,MQAKILRIATRKSPLAICQACYVCNKLKHYHPHIQTELIPIITTGD...,"[0.0707395498392283, 0.02572347266881029, 0.04...","[0.0, 0.0032258064516129032, 0.003225806451612...",35011.6433,9.364049,-0.041479,0.051447,38.157235,116.720257,0.359100
1,MKKLRIGARDSKLSRIQVDIVARKIKQTLGIECEFVPIKTKGDIDK...,"[0.05517241379310345, 0.02413793103448276, 0.0...","[0.0034602076124567475, 0.0, 0.003460207612456...",32275.0348,8.347963,-0.154483,0.058621,42.982793,105.862069,0.291586
2,MKKELIIGTRSSPLALWQAEFTKAELSRHFPELNITLKLVKTTGDV...,"[0.07051282051282051, 0.01282051282051282, 0.0...","[0.0, 0.0, 0.003215434083601286, 0.02250803858...",34393.4749,6.042860,-0.122436,0.044872,40.237821,101.923077,0.262981
3,MFNRQASGGAGSSGQGAGSSQTASAAPVSAGVGVGGGGGASGAAAG...,"[0.09324104234527687, 0.023615635179153095, 0....","[0.010792099368764, 0.0030543677458766036, 0.0...",529990.8042,6.424364,-0.162032,0.053542,46.634426,90.848941,0.260615
4,MPSESFCLAAQARLDSKWLKTDIQLAFTRDGLCGLWNEMVKDGEIV...,"[0.07840297889946214, 0.02399669011170873, 0.0...","[0.008069522036002483, 0.0028967515001034555, ...",527221.8439,5.875981,-0.186202,0.060612,42.951614,90.877120,0.276119
...,...,...,...,...,...,...,...,...,...,...
110217,MVRTRLAISVVLVSTLLLLNVKAKSVDPYKVLGVSKDAKQREIQKA...,"[0.050699300699300696, 0.008741258741258742, 0...","[0.0035026269702276708, 0.0, 0.0, 0.0070052539...",62569.2650,9.518709,-0.490559,0.089161,38.246521,72.237762,0.166154
110218,MPKAPKQQPPEPEWIGDGESTSPSDKVVKKGKKDKKIKKTFFEELA...,"[0.07558859975216853, 0.009913258983890954, 0....","[0.009925558312655087, 0.0012406947890818859, ...",91651.1398,7.208401,-0.883643,0.055762,47.106072,74.361834,0.030607
110219,MTDPHTARTIVGIVGNVISFGLFCAPIPTMVKIWKMKSVSEFKPDP...,"[0.05, 0.020833333333333332, 0.033333333333333...","[0.0041841004184100415, 0.0041841004184100415,...",27209.3824,8.883244,0.681667,0.137500,34.603792,111.583333,0.752250
110220,MAVPASPQHPRGYGILLLTLLLKALATTASACNHLRPQDATFSHDS...,"[0.07772020725388601, 0.031088082901554404, 0....","[0.0, 0.010416666666666666, 0.0052083333333333...",22115.7405,9.055116,-0.532124,0.067358,65.206218,81.502591,0.197876


In [5]:
train_df = pd.read_csv("../../dataset/ec40/train.csv")
valid_df = pd.read_csv("../../dataset/ec40/valid.csv")
test_df = pd.read_csv("../../dataset/ec40/test.csv")

In [7]:
fe_train = fe_df[fe_df['Original'].isin(train_df['sequence'])]
fe_test  = fe_df[fe_df['Original'].isin(test_df['sequence'])]
fe_valid = fe_df[fe_df['Original'].isin(valid_df['sequence'])]
train_df = train_df.merge(fe_train, left_on='sequence', right_on='Original', how='left').drop(columns=['Original', 'traintest', 'negative_for', 'mainclass_set', 'sprot_version', 'len', 'cluster_ID', 'representative', 'sequence', 'accession'])
test_df  = test_df.merge(fe_test, left_on='sequence', right_on='Original', how='left').drop(columns=['Original', 'traintest', 'negative_for', 'mainclass_set', 'sprot_version', 'len', 'cluster_ID', 'representative', 'sequence', 'accession'])
valid_df = valid_df.merge(fe_valid, left_on='sequence', right_on='Original', how='left').drop(columns=['Original', 'traintest', 'negative_for', 'mainclass_set', 'sprot_version', 'len', 'cluster_ID', 'representative', 'sequence', 'accession'])

In [8]:

def encode_ec_vector(x):
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                x = parsed[0]
            else:
                x = parsed
        except Exception:
            pass
    if isinstance(x, str):
        parts = x.split('.')
    else:
        parts = []

    vec = []
    for part in parts:
        part = part.strip()
        if part == '-' or part == '':
            vec.append(-1)
        else:
            try:
                vec.append(int(part))
            except ValueError:
                try:
                    vec.append(float(part))
                except Exception:
                    vec.append(-1)
    return vec

def convert_str_to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return x
    return x

def expand_column(df, column_names, drop_original=False):

    for column_name in column_names:
        expanded_cols = df[column_name].apply(pd.Series)
        expanded_cols.columns = [f"{column_name}_{i}" for i in expanded_cols.columns]
        df = pd.concat([df, expanded_cols], axis=1)
        if drop_original:
            df = df.drop(columns=[column_name])
    return df

In [9]:
train_df['ec'] = train_df['ec'].apply(encode_ec_vector)
valid_df['ec'] = valid_df['ec'].apply(encode_ec_vector)
test_df['ec'] = test_df['ec'].apply(encode_ec_vector)

In [11]:
cols_to_convert = ['AAC', 'DC']
for col in cols_to_convert:
    train_df[col] = train_df[col].apply(convert_str_to_list)
    valid_df[col] = valid_df[col].apply(convert_str_to_list)
    test_df[col] = test_df[col].apply(convert_str_to_list)

In [12]:
train_df = expand_column(train_df, ['ec', 'AAC', 'DC'], True)
valid_df = expand_column(valid_df, ['ec', 'AAC', 'DC'], True)
test_df = expand_column(test_df, ['ec', 'AAC', 'DC'], True)

In [14]:
train_df.to_csv("../../dataset/physiochemical/train.csv", index=False)
test_df.to_csv("../../dataset/physiochemical/test.csv", index=False)
valid_df.to_csv("../../dataset/physiochemical/valid.csv", index=False)

## Loading data

In [ ]:
train_df = pd.read_csv("../../dataset/physiochemical/train.csv")
valid_df = pd.read_csv("../../dataset/physiochemical/valid.csv")
test_df = pd.read_csv("../../dataset/physiochemical/test.csv")

In [72]:
train_df

,ec_0,ec_1,ec_2,ec_3,AAC_0,AAC_1,AAC_2,AAC_3,AAC_4,AAC_5,...,KMERC_5804,KMERC_5805,KMERC_5806,KMERC_5807,KMERC_5808,KMERC_5809,KMERC_5810,KMERC_5811,KMERC_5812,KMERC_5813
0,2,5,1,61,0.055172,0.024138,0.082759,0.058621,0.037931,0.062069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,2,26,0.093241,0.023616,0.042752,0.057818,0.021173,0.090798,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,2,26,0.078403,0.023997,0.058130,0.061647,0.026479,0.081920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5,1,83,0.070769,0.012308,0.080000,0.055385,0.036923,0.043077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,3,1,78,0.084465,0.021116,0.039216,0.037707,0.046757,0.078431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45755,3,6,4,13,0.051095,0.025547,0.047445,0.054745,0.040146,0.043796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45756,3,6,4,12,0.037410,0.017266,0.056115,0.064748,0.069065,0.031655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45757,3,1,3,5,0.090090,0.013514,0.081081,0.058559,0.027027,0.054054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45758,3,1,4,-1,0.076923,0.017751,0.059172,0.094675,0.023669,0.076923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
valid_df

,ec_0,ec_1,ec_2,ec_3,AAC_0,AAC_1,AAC_2,AAC_3,AAC_4,AAC_5,...,KMERC_3962,KMERC_3963,KMERC_3964,KMERC_3965,KMERC_3966,KMERC_3967,KMERC_3968,KMERC_3969,KMERC_3970,KMERC_3971
0,2,4,2,17,0.119863,0.003425,0.061644,0.061644,0.030822,0.089041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4,2,17,0.119863,0.003425,0.061644,0.061644,0.030822,0.089041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,4,2,17,0.081272,0.007067,0.067138,0.116608,0.017668,0.081272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,6,1,9,0.095109,0.024457,0.048913,0.065217,0.029891,0.043478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,6,1,9,0.153425,0.019178,0.046575,0.041096,0.019178,0.052055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7306,3,4,23,-1,0.057922,0.008518,0.073254,0.052811,0.064736,0.022147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7307,3,6,3,-1,0.088353,0.004016,0.044177,0.088353,0.032129,0.064257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7308,3,1,3,-1,0.078818,0.004926,0.029557,0.029557,0.088670,0.068966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7309,3,1,-1,-1,0.049296,0.000000,0.049296,0.056338,0.056338,0.035211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
test_df

,ec_0,ec_1,ec_2,ec_3,AAC_0,AAC_1,AAC_2,AAC_3,AAC_4,AAC_5,...,KMERC_3200,KMERC_3201,KMERC_3202,KMERC_3203,KMERC_3204,KMERC_3205,KMERC_3206,KMERC_3207,KMERC_3208,KMERC_3209
0,3,6,1,41,0.069343,0.021898,0.080292,0.040146,0.036496,0.054745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1,3,48,0.044610,0.017348,0.049566,0.073110,0.044610,0.030979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6,4,13,0.100391,0.007823,0.080834,0.048240,0.031291,0.041721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6,4,-1,0.065924,0.021820,0.048282,0.074280,0.027855,0.067781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1,26,3,0.060284,0.000000,0.053191,0.078014,0.070922,0.042553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8176,2,7,1,176,0.045296,0.000000,0.041812,0.087108,0.027875,0.052265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8177,2,4,1,-1,0.041758,0.015385,0.043956,0.076923,0.048352,0.072527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8178,2,1,1,182,0.098305,0.000000,0.050847,0.054237,0.030508,0.044068,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8179,2,7,7,4,0.092683,0.014634,0.058537,0.075610,0.036585,0.085366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Processing

In [73]:
patterns = ['AAC', 'DC', 'TC', 'KMERC']
cols = [col for col in train_df.columns if any(pattern in col for pattern in patterns)]

scaler = MinMaxScaler()
scaler.fit(train_df[cols])
train_df[cols] = scaler.transform(train_df[cols])
valid_df[cols] = scaler.transform(valid_df[cols])
test_df[cols]  = scaler.transform(test_df[cols])

KeyError: "['TC_3972', 'TC_3973', 'TC_3974', 'TC_3975', 'TC_3976', 'TC_3977', 'TC_3978', 'TC_3979', 'TC_3980', 'TC_3981', 'TC_3982', 'TC_3983', 'TC_3984', 'TC_3985', 'TC_3986', 'TC_3987', 'TC_3988', 'TC_3989', 'TC_3990', 'TC_3991', 'TC_3992', 'TC_3993', 'TC_3994', 'TC_3995', 'TC_3996', 'TC_3997', 'TC_3998', 'TC_3999', 'TC_4000', 'TC_4001', 'TC_4002', 'TC_4003', 'TC_4004', 'TC_4005', 'TC_4006', 'TC_4007', 'TC_4008', 'TC_4009', 'TC_4010', 'TC_4011', 'TC_4012', 'TC_4013', 'TC_4014', 'TC_4015', 'TC_4016', 'TC_4017', 'TC_4018', 'TC_4019', 'TC_4020', 'TC_4021', 'TC_4022', 'TC_4023', 'TC_4024', 'TC_4025', 'TC_4026', 'TC_4027', 'TC_4028', 'TC_4029', 'TC_4030', 'TC_4031', 'TC_4032', 'TC_4033', 'TC_4034', 'TC_4035', 'TC_4036', 'TC_4037', 'TC_4038', 'TC_4039', 'TC_4040', 'TC_4041', 'TC_4042', 'TC_4043', 'TC_4044', 'TC_4045', 'TC_4046', 'TC_4047', 'TC_4048', 'TC_4049', 'TC_4050', 'TC_4051', 'TC_4052', 'TC_4053', 'TC_4054', 'TC_4055', 'TC_4056', 'TC_4057', 'TC_4058', 'TC_4059', 'TC_4060', 'TC_4061'